# Last FM hometask <br>
https://www.kaggle.com/ravichaubey1506/lastfm <br>
1. Выбрать данные по странам своей группы (совместно): <br>
    3530203_70101: Germany, Netherlands <br>
    3530203_70102: Belarus, Ukraine, Poland, Russian Federation<br>
    3530903_70301: Sweden, Finland, Norway, Denmark, Iceland<br>
    3530903_70302: Spain, Portugal, France, Italy, Belgium<br>
    
2. Попытаться найти полезные с точки зрения продвижения групп (или еще чего-нибудь) и нетривиальные правила, используя алгоритмы Apriori, FPGrowth, FPMax и всевозможные метрики. Хотя бы 5 правил.
3. Вывести эти правила в отдельных ячейках. 
4. Подумать, как можно было бы использовать полученные правила.

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("lastfm.csv")
data.head()

,user,artist,sex,country
0,1,red hot chili peppers,f,Germany
1,1,the black dahlia murder,f,Germany
2,1,goldfrapp,f,Germany
3,1,dropkick murphys,f,Germany
4,1,le tigre,f,Germany


In [3]:
data = data[data.country.isin(['Spain', 'Portugal', 'France', 'Italy', 'Belgium'])]
data.head()

,user,artist,sex,country
83,6,lily allen,m,Portugal
84,6,kanye west,m,Portugal
85,6,sigur rós,m,Portugal
86,6,pink floyd,m,Portugal
87,6,stevie wonder,m,Portugal


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27214 entries, 83 to 289762
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user     27214 non-null  int64 
 1   artist   27214 non-null  object
 2   sex      27214 non-null  object
 3   country  27214 non-null  object
dtypes: int64(1), object(3)
memory usage: 1.0+ MB


In [5]:
data = data.drop('country', axis=1).groupby(['user','sex'])['artist'].apply(lambda x: ';'.join(x)).reset_index()
data

,user,sex,artist
0,6,m,lily allen;kanye west;sigur rós;pink floyd;ste...
1,12,m,queen;the beatles;lynyrd skynyrd;neil young;th...
2,36,f,m.i.a.;fugazi;johnny cash;clint mansell;elton ...
3,47,m,pj harvey;radiohead;zero 7;einstürzende neubau...
4,74,m,in extremo;nine inch nails;billy talent;panic!...
...,...,...,...
1414,19690,m,pink floyd;the beatles;the white stripes;depec...
1415,19693,m,air;simon & garfunkel;pixies;serge gainsbourg;...
1416,19697,m,underworld;pendulum;the prodigy;sufjan stevens...
1417,19704,m,keane;pinback;coldplay;travis;calexico;doves;d...


In [6]:
pvt = pd.concat([data.drop(['sex','artist'], axis=1), 
                             pd.get_dummies(data['sex']),
                             data['artist'].str.get_dummies(';')
                            ], axis = 1).set_index(['user'])
pvt

,f,m,...and you will know us by the trail of dead,2pac,3 doors down,30 seconds to mars,311,36 crazyfists,44,50 cent,...,wilco,within temptation,wolfgang amadeus mozart,wu-tang clan,yann tiersen,yeah yeah yeahs,yellowcard,yo la tengo,zero 7,Édith piaf
user,,,,,,,,,,,,,,,,,,,,,
6,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
36,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
47,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
74,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19690,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19693,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19697,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
from mlxtend.frequent_patterns import apriori, association_rules
frequent_itemsets = apriori(pvt, min_support=0.02, use_colnames=True)\
                        .sort_values(by='support', ascending=False, na_position='last')
frequent_itemsets

,support,itemsets
1,0.780127,(m)
211,0.237491,(radiohead)
0,0.219873,(f)
443,0.193094,"(m, radiohead)"
62,0.188161,(coldplay)
...,...,...
537,0.020437,"(the white stripes, arctic monkeys)"
982,0.020437,"(m, the killers, the beatles)"
731,0.020437,"(the doors, muse)"
980,0.020437,"(m, the beatles, the cure)"


In [8]:
rules_confidence = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.3)\
.sort_values(by='confidence', ascending=False, na_position='last')
rules_confidence

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
625,"(metallica, radiohead)",(m),0.024665,0.780127,0.024665,1.000000,1.281843,0.005423,inf
328,"(metallica, pink floyd)",(m),0.031712,0.780127,0.031008,0.977778,1.253357,0.006268,9.894292
449,"(metallica, led zeppelin)",(m),0.028894,0.780127,0.028189,0.975610,1.250578,0.005648,9.014799
689,(frank zappa),(m),0.024665,0.780127,0.023961,0.971429,1.245219,0.004719,7.695560
785,(isis),(m),0.023961,0.780127,0.023256,0.970588,1.244142,0.004564,7.475687
...,...,...,...,...,...,...,...,...,...
870,(the rolling stones),"(m, radiohead)",0.072586,0.193094,0.021846,0.300971,1.558678,0.007830,1.154324
718,(interpol),(placebo),0.079634,0.098661,0.023961,0.300885,3.049684,0.016104,1.289257
719,(interpol),(the cure),0.079634,0.103594,0.023961,0.300885,2.904461,0.015711,1.282201
949,"(radiohead, the beatles)",(led zeppelin),0.070472,0.098661,0.021142,0.300000,3.040714,0.014189,1.287627


In [9]:
# нет цели определять пол по списку исполнителей
rules_confidence2 = rules_confidence[(rules_confidence['consequents'] != frozenset('m')) 
                                     & (rules_confidence['consequents'] != frozenset('f'))
                                    & (rules_confidence['consequent support'] < 0.1)]
rules_confidence2

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
1067,"(the killers, kaiser chiefs)",(franz ferdinand),0.031008,0.095842,0.020437,0.659091,6.876838,0.017465,2.652196
875,"(the killers, bloc party)",(the strokes),0.035236,0.087385,0.021846,0.620000,7.095000,0.018767,2.401617
769,"(the strokes, muse)",(interpol),0.038055,0.079634,0.023256,0.611111,7.674041,0.020225,2.366657
367,(kaiser chiefs),(franz ferdinand),0.050740,0.095842,0.030303,0.597222,6.231311,0.025440,2.244806
1019,"(m, editors)",(interpol),0.035236,0.079634,0.020437,0.580000,7.283363,0.017631,2.191349
...,...,...,...,...,...,...,...,...,...
884,"(m, franz ferdinand)",(kaiser chiefs),0.071882,0.050740,0.021846,0.303922,5.989788,0.018199,1.363726
1064,"(m, bloc party)",(franz ferdinand),0.067653,0.095842,0.020437,0.302083,3.151884,0.013953,1.295510
1083,"(m, moby)",(massive attack),0.067653,0.088090,0.020437,0.302083,3.429250,0.014477,1.306617
718,(interpol),(placebo),0.079634,0.098661,0.023961,0.300885,3.049684,0.016104,1.289257


In [10]:
rules_lift = association_rules(frequent_itemsets, metric="lift", min_threshold=0.3).sort_values(by='lift', ascending=False, na_position='last')
rules_lift

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
2411,(kaiser chiefs),"(the killers, franz ferdinand)",0.050740,0.040169,0.020437,0.402778,10.027047,0.018399,1.607159
2406,"(the killers, franz ferdinand)",(kaiser chiefs),0.040169,0.050740,0.020437,0.508772,10.027047,0.018399,1.932422
1994,(megadeth),(iron maiden),0.038760,0.073291,0.021846,0.563636,7.690385,0.019006,2.123708
1995,(iron maiden),(megadeth),0.073291,0.038760,0.021846,0.298077,7.690385,0.019006,1.369438
1664,(interpol),"(the strokes, muse)",0.079634,0.038055,0.023256,0.292035,7.674041,0.020225,1.358747
...,...,...,...,...,...,...,...,...,...
1505,(fall out boy),(m),0.038760,0.780127,0.023961,0.618182,0.792412,-0.006277,0.575858
1612,(m),(norah jones),0.780127,0.038055,0.023256,0.029810,0.783348,-0.006432,0.991502
1613,(norah jones),(m),0.038055,0.780127,0.023256,0.611111,0.783348,-0.006432,0.565388
2414,(m),(britney spears),0.780127,0.034531,0.020437,0.026197,0.758642,-0.006502,0.991441


In [11]:
rules_confidence2.head(1)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
1067,"(the killers, kaiser chiefs)",(franz ferdinand),0.031008,0.095842,0.020437,0.659091,6.876838,0.017465,2.652196


In [12]:
rules_lift.head(1)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
2411,(kaiser chiefs),"(the killers, franz ferdinand)",0.05074,0.040169,0.020437,0.402778,10.027047,0.018399,1.607159


In [13]:
rules_leverage = association_rules(frequent_itemsets, metric="leverage", min_threshold=0.01).sort_values(by='leverage', ascending=False, na_position='last')
rules_leverage

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
6,(muse),(radiohead),0.174066,0.237491,0.080338,0.461538,1.943392,0.038999,1.416088
7,(radiohead),(muse),0.237491,0.174066,0.080338,0.338279,1.943392,0.038999,1.248160
4,(coldplay),(radiohead),0.188161,0.237491,0.083157,0.441948,1.860901,0.038471,1.366375
5,(radiohead),(coldplay),0.237491,0.188161,0.083157,0.350148,1.860901,0.038471,1.249269
10,(coldplay),(muse),0.188161,0.174066,0.068358,0.363296,2.087113,0.035606,1.297202
...,...,...,...,...,...,...,...,...,...
1190,(muse),"(m, moby)",0.174066,0.067653,0.021846,0.125506,1.855137,0.010070,1.066156
887,"(m, foo fighters)",(radiohead),0.058492,0.237491,0.023961,0.409639,1.724858,0.010069,1.291597
888,(radiohead),"(m, foo fighters)",0.237491,0.058492,0.023961,0.100890,1.724858,0.010069,1.047156
308,(f),(placebo),0.219873,0.098661,0.031712,0.144231,1.461882,0.010020,1.053250


In [14]:
rules_conviction = association_rules(frequent_itemsets, metric="conviction", min_threshold=0.01).sort_values(by='conviction', ascending=False, na_position='last')
rules_conviction

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
1304,"(metallica, radiohead)",(m),0.024665,0.780127,0.024665,1.000000,1.281843,0.005423,inf
1135,"(m, keane)",(coldplay),0.028189,0.188161,0.026075,0.925000,4.916011,0.020771,10.824524
644,"(metallica, pink floyd)",(m),0.031712,0.780127,0.031008,0.977778,1.253357,0.006268,9.894292
918,"(metallica, led zeppelin)",(m),0.028894,0.780127,0.028189,0.975610,1.250578,0.005648,9.014799
1459,(frank zappa),(m),0.024665,0.780127,0.023961,0.971429,1.245219,0.004719,7.695560
...,...,...,...,...,...,...,...,...,...
2375,(beyoncé),(m),0.032417,0.780127,0.020437,0.630435,0.808118,-0.004853,0.594951
359,(the kooks),(m),0.058492,0.780127,0.036646,0.626506,0.803082,-0.008986,0.588693
1505,(fall out boy),(m),0.038760,0.780127,0.023961,0.618182,0.792412,-0.006277,0.575858
1613,(norah jones),(m),0.038055,0.780127,0.023256,0.611111,0.783348,-0.006432,0.565388


In [15]:
rules_confidence[(rules_confidence['consequents'] != frozenset('m')) 
                                     & (rules_confidence['consequents'] != frozenset('f'))
                                      & (rules_confidence['consequent support'] < 0.08)
                                       & ( rules_confidence['antecedent support'] > 0.1)
                                    ].head(1)
# слушателям популярной металлики можно попробовать послушать менее популярных айрон мейден

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
189,"(m, metallica)",(iron maiden),0.109232,0.073291,0.036646,0.335484,4.577419,0.02864,1.394562


In [16]:
rules_lift2 = rules_lift[(rules_lift['consequents'] != frozenset('m'))
                         & (rules_lift['consequents'] != frozenset('f'))
                        & (rules_lift['consequent support'] < 0.1)
                        ].sort_values(by='lift', ascending=True, na_position='last')
rules_lift2.head(1)
# не надо продвигать бритни спирс на мужскую аудиторию. им не заходит(

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
2414,(m),(britney spears),0.780127,0.034531,0.020437,0.026197,0.758642,-0.006502,0.991441


In [17]:
rules_leverage2 = rules_leverage[(rules_leverage['consequents'] != frozenset('m'))
                                 & (rules_leverage['consequents'] != frozenset('f'))
                                  & (rules_leverage['consequent support'] < 0.1)
                                  & (rules_leverage['antecedent support'] < 0.1)
                                ]
rules_leverage2.head(2)
#объединяем аулиторию строукс и интерпола

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
249,(interpol),(the strokes),0.079634,0.087385,0.033827,0.424779,4.860976,0.026868,1.586545
248,(the strokes),(interpol),0.087385,0.079634,0.033827,0.387097,4.860976,0.026868,1.501651
